<a id='table_of_contents'></a>

0. [Import libraries](#imports)
1. [Import data](#import_data)
2. [Initial Cleaning](#initial_cleaning)
3. [Price and Quantity Cleaning](#price_and_quantity_cleaning)
4. [Data Checks](#data_checks)
5. [Export cleaned data](#export_data)

# 0. Import libraries <a id='imports'></a>
[Back to top](#table_of_contents)

In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import json

import pandas as pd
import numpy as np
from ipywidgets import VBox, HBox
import matplotlib.pyplot as plt
import matplotlib as mpl

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_colwidth", 100)
mpl.rcParams["figure.dpi"] = 300

# 1. Import raw data <a id='import_data'></a>
[Back to top](#table_of_contents)

In [2]:
# Read in raw data
BASE_DIR = Path().resolve().parent
DATA_DIR = BASE_DIR / "data"

df_in = pd.read_csv(DATA_DIR / "raw" / "25072024_reviews.csv")
df_in.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7890 entries, 0 to 7889
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   rating                             7890 non-null   object 
 1   roaster                            7890 non-null   object 
 2   title                              7890 non-null   object 
 3   blind_assessment                   7889 non-null   object 
 4   notes                              7888 non-null   object 
 5   bottom_line                        3812 non-null   object 
 6   roaster location                   7887 non-null   object 
 7   coffee origin                      7386 non-null   object 
 8   roast level                        7488 non-null   object 
 9   agtron                             7890 non-null   object 
 10  est. price                         5852 non-null   object 
 11  review date                        7890 non-null   objec

# 2. Initial Cleaning <a id='initial_cleaning'></a>
[Back to top](#table_of_contents)

First step is to do some of the basic data checks and cleanup. This includes dropping columns that are not needed, setting datatypes, renaming columns,
combining columns, cleaning up strings, and creating new columns. 

In [3]:
# Cleanup column names
df_in.columns = (
    df_in.columns.str.strip().str.lower().str.replace(" ", "_").str.replace(".", "")
)

df_in.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7890 entries, 0 to 7889
Data columns (total 21 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   rating                            7890 non-null   object 
 1   roaster                           7890 non-null   object 
 2   title                             7890 non-null   object 
 3   blind_assessment                  7889 non-null   object 
 4   notes                             7888 non-null   object 
 5   bottom_line                       3812 non-null   object 
 6   roaster_location                  7887 non-null   object 
 7   coffee_origin                     7386 non-null   object 
 8   roast_level                       7488 non-null   object 
 9   agtron                            7890 non-null   object 
 10  est_price                         5852 non-null   object 
 11  review_date                       7890 non-null   object 
 12  aroma 

In [4]:
def tweak_df(df: pd.DataFrame) -> pd.DataFrame:
    """Initial data cleaning and feature creation"""
    return (
        df.assign(
            review_date=lambda df_: pd.to_datetime(df_["review_date"], format="%B %Y"),
            # Combing acidity and acidity/structure into one column, they are the same
            # field but names used in reviews changed at one point
            acidity=lambda df_: df_["acidity"].fillna(df_["acidity/structure"]),
            # Split the agtron column into one for external bean agtron data and ground
            # bean agtron data
            agtron_external=lambda df_: pd.to_numeric(
                df_["agtron"].str.split("/", expand=True)[0].str.strip(),
                errors="coerce",
            ),
            agtron_ground=lambda df_: pd.to_numeric(
                df_["agtron"].str.split("/", expand=True)[1].str.strip(),
                errors="coerce",
            ),
            # Distinguish espresso roasts from other reviews
            is_espresso=lambda df_: df_.apply(
                lambda row: (
                    True
                    if "espresso" in row["title"].lower()
                    or pd.notnull(row["with_milk"])
                    else False
                ),
                axis=1,
            ),
        )
        .replace(["", "NR", "N/A", "na"], np.nan)
        # Agtron values must be equalt to or below 100, some entries on website have typos
        .loc[
            lambda df_: (df_["agtron_external"] <= 100) & (df_["agtron_ground"] <= 100),
            :,
        ]
        # Run str.strip on every string column
        .map(lambda x: x.strip() if isinstance(x, str) else x)
        .drop(
            columns=["acidity/structure", "agtron", "refresh(enable_javascript_first)"]
        )
        .astype(
            {
                k: "float"
                for k in [
                    "agtron_external",
                    "agtron_ground",
                    "acidity",
                    "rating",
                    "aroma",
                    "body",
                    "flavor",
                    "aftertaste",
                ]
            }
        )
    )


df = tweak_df(df_in)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7566 entries, 0 to 7889
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   rating            7563 non-null   float64       
 1   roaster           7566 non-null   object        
 2   title             7566 non-null   object        
 3   blind_assessment  7566 non-null   object        
 4   notes             7565 non-null   object        
 5   bottom_line       3758 non-null   object        
 6   roaster_location  7564 non-null   object        
 7   coffee_origin     7299 non-null   object        
 8   roast_level       7445 non-null   object        
 9   est_price         5790 non-null   object        
 10  review_date       7566 non-null   datetime64[ns]
 11  aroma             7540 non-null   float64       
 12  acidity           6322 non-null   float64       
 13  body              7556 non-null   float64       
 14  flavor            7553 non-nu

# 3. Price and Quantity Cleaning <a id='price_and_quantity_cleaning'></a>
[Back to top](#table_of_contents)

The `est_price` column contains information on price, currency, and quantity. We need to split this column up to separate the price and quantity information.

Splitting on "/" character creates one column with price and currency information and another with quantity and unit of measurement information. 

The quantities have to be standardized so they contain a single representation for each unit and so unecessary punctuation and parentheses are removed.

We also filter the dataset to remove all products that came in units of cans, boxes, capusles, pods, etc. We will only concern ourselves with coffee sold in bags or bulk, ground or whole.


### Cleaning Quantity

In [5]:
drop_terms = [
    "can",
    "box",
    "capsules",
    "K-",
    "cups",
    "bags",
    "concentrate",
    "discs",
    "bottle",
    "pods",
    "ml",
    "pods",
    "pouch",
    "packet|tin",
    "instant",
    "sachet",
    "vue",
    "single-serve",
    "fluid",
    "capsultes",
]

drop_terms_string = "|".join(drop_terms)


def price_quantity_split(df: pd.DataFrame) -> pd.DataFrame:
    print(f"Original df shape: {df.shape}")
    price_quantity = (
        df
        # Split est_price into columns for price and quantity
        .est_price.str.split("/", n=1, expand=True)
        # Remove any commas from the price and quantity columns
        .replace(",", "", regex=True)
        .rename(columns={0: "price", 1: "quantity"})
        .assign(
            # Cleanup quantity
            quantity=lambda df_: (
                df_["quantity"]
                .str.lower()
                .str.strip()
                # Remove parentheses and anything inside them
                .str.replace(r"\(.*?\)", "", regex=True)
                # Remove anything after a semicolon. This is usually a note, or deal price.
                .str.replace(r";.*", "", regex=True)
                # Standardize units
                .str.replace(r".g$", " grams", regex=True)
                .str.replace(r"\sg$", "grams", regex=True)
                .str.replace(r"\bgram$", "grams", regex=True)
                .str.replace(r"pound$", "1 pounds", regex=True)
                .str.replace(r"oz|onces|ouncues|ounce$|ounces\*", "ounces", regex=True)
                .str.replace("kilogram", "kilograms")
                .str.replace("kg", "kilograms")
                # Remove "online" from any quantity
                .str.replace("online", "")
                .str.strip()
            ),
            price=lambda df_: df_["price"].str.replace("..", "."),
        )
        .dropna()
        # Remove rows where coffee is sold in a can, box, pouch, packet, or tin
        .loc[
            lambda df_: ~df_["quantity"].str.contains(drop_terms_string, case=False),
            :,
        ]
        # Split quantity into value and unit, and split price into value and currency
        .assign(
            # Extract number value from quantity
            quantity_value=lambda df_: (
                df_["quantity"].str.extract(r"(\d+)").astype(float)
            ),
            # Extract the unit from quantity column
            quantity_unit=lambda df_: (
                df_["quantity"]
                .str.replace(r"(\d+)", "", regex=True)
                .replace("\.", "", regex=True)
                .str.strip()
                .mask(lambda s: s == "g", "grams")
                .mask(lambda s: s == "kilo", "kilograms")
                .str.strip()
            ),
            # Extract price value from price column
            price_value=lambda df_: (
                df_["price"].str.extract(r"(\d+\.\d+|\d+)").astype(float)
            ),
            # Extract currency from price column
            price_currency=lambda df_: (
                df_["price"]
                .str.replace(",", "")
                .str.replace(r"(\d+\.\d+|\d+)", "", regex=True)
                .str.strip()
            ),
        )
        # Drop the original price and quantity columns
        .drop(columns=["price", "quantity"])
        # remove rows where quantity_unit contains (
        .loc[lambda df_: ~df_["quantity_unit"].str.contains(r"\(", regex=True), :]
    )
    print(f"Shape of price_quantity: {price_quantity.shape}")

    # Merge the price_quantity DataFrame with the original DataFrame
    return df.merge(price_quantity, how="left", left_index=True, right_index=True)


df = df_in.pipe(tweak_df).pipe(price_quantity_split)

df.quantity_unit.value_counts()

Original df shape: (7566, 21)
Shape of price_quantity: (5607, 4)


quantity_unit
ounces       4757
grams         824
pounds         17
kilograms       9
Name: count, dtype: int64

### Cleaning Prices and Currencies
Normalize the currency column to contain a standardized set of currency symbols. We will use the ISO 4217 codes to make it easier to get foreign exchange data from an external API later on. 


In [45]:
def clean_currency(df: pd.DataFrame) -> pd.DataFrame:
    """Standardize currencies to ISO 4217 codes."""
    price_currency = (
        df.price_currency.str.upper()
        .str.replace(r"^\$$", "USD", regex=True)
        .str.replace("PRICE: $", "USD")
        .str.replace("$", "")
        .str.replace("#", "GBP")
        .str.replace("¥", "JPY")
        .str.replace("£", "GBP")
        .str.replace("€", "EUR")
        .str.replace("POUND", "GBP")
        .str.replace("PESOS", "MXN")
        .str.replace("RMB", "CNY")
        .str.replace("EUROS", "EUR")
        .str.replace("RM", "MYR")
        .str.strip()
        .mask(lambda s: s == "US", "USD")
        .mask(lambda s: s == " ", "USD")
        .mask(lambda s: s == "E", "EUR")
        .mask(lambda s: s == "NTD", "TWD")
        .mask(lambda s: s == "NT", "TWD")
        .mask(lambda s: s == "", "USD")
        .mask(lambda s: s == "HK", "HKD")
        .str.strip()
    )
    return df.assign(price_currency=price_currency)


df = df_in.pipe(tweak_df).pipe(price_quantity_split).pipe(clean_currency)


# Check that currencies make sense from original est_price column
df.loc[:, ["est_price", "price_currency", "price_value"]].groupby(
    "price_currency"
).sample(3, replace=True)

Original df shape: (7566, 21)
Shape of price_quantity: (5607, 4)


,est_price,price_currency,price_value
7560,AED $99.75/250 grams,AED,99.75
5351,AED $103.95/250 grams,AED,103.95
7560,AED $99.75/250 grams,AED,99.75
7827,AUD $18.00/18 grams,AUD,18.00
6014,AUD $23.00/400 grams,AUD,23.00
6014,AUD $23.00/400 grams,AUD,23.00
6174,CAD $15 / 10.6 oz.,CAD,15.00
1193,CAD $15.99/16 ounces,CAD,15.99
5179,CAD $11.50 / 16 oz.,CAD,11.50
2992,RMB $80.00/454 grams,CNY,80.00


In [46]:
df.price_currency.value_counts()

price_currency
USD    4249
TWD    1067
CAD     125
HKD      46
CNY      27
THB      21
KRW      20
JPY      12
GBP      10
AUD      10
EUR       6
AED       5
MYR       3
IDR       3
GTQ       1
MXN       1
LAK       1
Name: count, dtype: int64

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7566 entries, 0 to 7889
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   rating            7563 non-null   float64       
 1   roaster           7566 non-null   object        
 2   title             7566 non-null   object        
 3   blind_assessment  7566 non-null   object        
 4   notes             7565 non-null   object        
 5   bottom_line       3758 non-null   object        
 6   roaster_location  7564 non-null   object        
 7   coffee_origin     7299 non-null   object        
 8   roast_level       7445 non-null   object        
 9   est_price         5790 non-null   object        
 10  review_date       7566 non-null   datetime64[ns]
 11  aroma             7540 non-null   float64       
 12  acidity           6322 non-null   float64       
 13  body              7556 non-null   float64       
 14  flavor            7553 non-nu

#### Converting prices to 2024 USD

1. Convert price to USD using historical exchange rates
2. Adjust price to 2024 USD using BLS consumer price index

In [47]:
with open(DATA_DIR / "external" / "openex_exchange_rates.json") as f:
    currency_codes = json.load(f)


def convert_currency(row):
    try:
        date = row.review_date.strftime("%Y-%m-%d")
        currency = row.price_currency
        value = np.round(row.price_value / currency_codes[date][currency], 2)
    except KeyError:
        value = np.nan
    return value


df = df_in.pipe(tweak_df).pipe(price_quantity_split).pipe(clean_currency)


df["price_usd"] = df.apply(convert_currency, axis=1)

df.groupby("price_currency")[
    [
        "price_usd",
        "price_value",
        "price_currency",
    ]
].sample(1)

Original df shape: (7566, 21)
Shape of price_quantity: (5607, 4)


,price_usd,price_value,price_currency
415,25.86,95.00,AED
1030,11.60,16.00,AUD
6487,14.46,14.05,CAD
1710,38.23,235.00,CNY
6557,67.95,50.00,EUR
1142,9.29,5.50,GBP
1575,12.53,100.00,GTQ
7386,17.95,140.00,HKD
6363,8.89,120000.00,IDR
6696,23.44,2420.00,JPY


In [56]:
def load_transform_cpi(file_path: Path) -> pd.DataFrame:
    """Loads and transforms the CPI data."""
    try:
        cpi = pd.read_csv(file_path)
    except FileNotFoundError:
        raise FileNotFoundError("CPI file is not found in the specified directory.")

    cpi.columns = cpi.columns.str.strip().str.lower().str.replace(" ", "_")
    return (
        cpi.drop(columns=["half1", "half2"])
        .melt(id_vars="year", var_name="month", value_name="cpi")
        .assign(
            month=lambda df_: df_["month"]
            .apply(lambda x: datetime.strptime(x, "%b").month)
            .dropna()
            .reset_index(drop=True)
        )
    )


cpi = load_transform_cpi(DATA_DIR / "external" / "consumer_price_index.csv")
cpi

NameError: name 'datetime' is not defined

In [ ]:
def create_cpi_adjusted_price(
    df: pd.DataFrame, file_path: Path, date: str = "2024-01-01"
) -> pd.DataFrame:
    """
    Adjusts historical price data to 2024 prices using CPI data.
    """
    cpi = load_transform_cpi(file_path)
    cpi_baseline = cpi.loc[cpi["date"] == date, "consumer_price_index"].values[0]

    return (
        df.merge(cpi, left_on="review_date", right_on="date")
        .drop(columns="date")
        .assign(
            price_usd_adj_2024=lambda df_: np.round(
                df_["price_value_usd_hist"]
                * cpi_baseline
                / df_["consumer_price_index"],
                2,
            )
        )
    )


cpi_path = DATA_DIR / "external" / "consumer_price_index.csv"

df = (
    df_in.pipe(tweak_df)
    .pipe(price_quantity_split)
    .pipe(clean_currency)
    .pipe(convert_to_usd)
    .pipe(create_cpi_adjusted_price, file_path=cpi_path)
)

df.sample(3)

In [9]:
(
    df.assign(
        price_diff=lambda df_: (df_["price_usd_adj_2024"] - df_["price_value_usd_hist"])
        / df_["price_usd_adj_2024"]
    )
).plot(
    x="review_date",
    y="price_diff",
    title="% Price difference between adjusted and historical prices",
)

KeyError: 'price_usd_adj_2024'

#### Converting quantities to lbs

Create a normalized quantity column that converts all quantities to lbs.

In [ ]:
def convert_to_lbs(df: pd.DataFrame) -> pd.DataFrame:
    to_lbs_conversion = {
        "ounces": 1 / 16,
        "pounds": 1,
        "kilogram": 2.20462,
        "grams": 0.00220462,
    }
    df["quantity_in_lbs"] = np.round(
        df["quantity_value"] * df["quantity_unit"].map(to_lbs_conversion), 2
    )
    return df


df = (
    df_raw.pipe(tweak_df)
    .pipe(price_quantity_split)
    .pipe(clean_currency)
    .pipe(convert_to_usd)
    .pipe(create_cpi_adjusted_price, file_path=cpi_path)
    .pipe(convert_to_lbs)
)

(
    df.loc[:, ["quantity_value", "quantity_unit", "quantity_in_lbs"]]
    .groupby("quantity_unit")
    .sample(3, replace=True)
)

In [ ]:
# Create a new column for price per pound
def price_per_lbs(df: pd.DataFrame) -> pd.DataFrame:
    df["price_usd_adj_2024_per_lb"] = np.round(
        df["price_usd_adj_2024"] / df["quantity_in_lbs"], 2
    )
    return df


df = (
    df_raw.pipe(tweak_df)
    .pipe(price_quantity_split)
    .pipe(clean_currency)
    .pipe(convert_to_usd)
    .pipe(create_cpi_adjusted_price, file_path=cpi_path)
    .pipe(convert_to_lbs)
    .pipe(price_per_lbs)
)

df.describe()

# 4. Data Checks <a id='data_checks'></a>
[Back to top](#table_of_contents)

# 5. Export cleaned data <a id='export_data'></a>
[Back to top](#table_of_contents)

In [ ]:
fout = data_dir / "processed" / "05052024_roast_review_cleaned.csv"
df.to_csv(fout, index=False)